In [36]:
import pandas as pd, os, datetime as dt

In [37]:
# read available imgs
img_files = os.listdir("../data/rsb-resized")
img_ts = [dt.datetime.strptime(img.split(".")[0], "%Y-%m-%d_%H-%M-%S") for img in img_files]
img_files = pd.DataFrame({"filename": img_files, "timestamp": img_ts})
img_files.sort_values("timestamp", inplace=True)
img_files.reset_index(drop=True, inplace=True)

# read target
pv_loc = "../data/stg-werkhof-interpolated-new.csv"
pv = pd.read_csv(pv_loc, sep=";", parse_dates=["from", "to"], encoding="utf-8")

# align pv and img data
start = max(img_files.timestamp.min(), pv["to"].min())
end = min(img_files.timestamp.max(), pv["to"].max())
img_files = img_files[(img_files.timestamp >= start) & (img_files.timestamp <= end)]
pv = pv[(pv["to"] >= start) & (pv["to"] <= end)]

# add has_image column
pv["has_image"] = pv["to"].apply(lambda x: x.to_pydatetime() in img_ts)

In [38]:
pv.head()

,from,to,energy-produced-Wh,has_image
58649,2024-02-12 07:50:00,2024-02-12 08:00:00,0.000000,True
58650,2024-02-12 08:00:00,2024-02-12 08:10:00,9.388000,True
58651,2024-02-12 08:10:00,2024-02-12 08:20:00,32.777333,True
58652,2024-02-12 08:20:00,2024-02-12 08:30:00,56.166667,True
58653,2024-02-12 08:30:00,2024-02-12 08:40:00,111.754667,True


In [39]:
pv['date'] = pv['from'].dt.date

# Identify the first record with has_image == True for each day
pv['is_first_with_image_that_day'] = pv.groupby(pv['from'].dt.date)['has_image'].transform(lambda x: x & (x.cumsum() == 1))

In [40]:
pv['is_last_with_image_that_day'] = pv.groupby(pv['from'].dt.date)['has_image'].transform(lambda x: x & (x[::-1].cumsum() == 1))

In [ ]:
for i, row in pv.iterrows():
    if row["is_first_with_image_that_day"]:
        atday = True

    if atday:
        pv.at[i, "atday"] = True
    else:
        pv.at[i, "atday"] = False

    if row["is_last_with_image_that_day"]:
        atday = False

In [50]:
((pv["atday"] == True) | (pv["atday"] == False)).all()

True

In [51]:
(pv["atday"] != None).all()

True

In [52]:
pv[~pv['atday'] & ~pv['has_image']].shape

(11803, 8)

In [53]:
pv[pv['atday'] & pv['has_image']].shape

(162, 8)

In [54]:
pv[~pv['atday'] & pv['has_image']].shape

(12606, 8)

In [55]:
pv[pv['atday'] & ~pv['has_image']].shape

(0, 8)

In [56]:
pv[pv['atday'] & ~pv['has_image']]

,from,to,energy-produced-Wh,has_image,date,is_first_with_image_that_day,is_last_with_image_that_day,atday
